## Useful functions

In [9]:
from graph_manipulation import *
from betwenness_computation import *
from error_measurements import *

## Used datasets

In [2]:
file_paths = [
    'datasets/socfb-American75/socfb-American75.mtx',
    'datasets/socfb-Auburn71/socfb-Auburn71.mtx'
]

## Iperparameters

In [3]:
classification_nodes = 20
approximation_rate = 0.3

## Graph loading

In [4]:
graphs = [create_graph_from_mtx(file_path) for file_path in file_paths]

Graph: datasets/socfb-American75/socfb-American75.mtx loaded
Number of nodes: 6386
Number of edges: 217663
Graph: datasets/socfb-Auburn71/socfb-Auburn71.mtx loaded
Number of nodes: 18448
Number of edges: 973919


### Exact betwenness calculation


**Algoritmo Utilizzato**: **Algoritmo di Brandes**

**Descrizione**:
- L'algoritmo di Brandes è l'algoritmo standard per il calcolo esatto della betweenness centrality. Questo algoritmo sfrutta una combinazione di attraversamenti in ampiezza del grafo e calcoli di somme per determinare il numero di percorsi più brevi che passano attraverso ciascun nodo.
- La complessità computazionale dell'algoritmo è \(O(nm)\) per grafi non pesati e \(O(nm + n^2 \log n)\) per grafi pesati, dove \(n\) è il numero di nodi e \(m\) è il numero di archi nel grafo. Questo lo rende efficiente per grafi di dimensioni moderate, ma può diventare oneroso per grafi molto grandi.

**Implementazione in NetworkX**:
- NetworkX utilizza l'algoritmo di Brandes per calcolare la betweenness centrality quando si utilizza la funzione `betweenness_centrality(graph)` senza parametri aggiuntivi.


In [5]:
exact_results = []
for i, g in enumerate(graphs):
    betweenness_exact, computation_time = calculate_betweenness_centrality(g)
    exact_results.append((betweenness_exact, computation_time))
    print(f"Graph {i+1} - Computation time: {computation_time} seconds")



Graph 1 - Computation time: 271.9077022075653 seconds
Graph 2 - Computation time: 3621.18812417984 seconds


### Betweenness Centrality Approssimata

**Algoritmo Utilizzato**: **Campionamento Randomizzato Basato su Algoritmo di Brandes**

**Descrizione**:
- Per approssimare la betweenness centrality, NetworkX implementa una versione modificata dell'algoritmo di Brandes che utilizza il campionamento randomizzato. Invece di calcolare i percorsi più brevi tra tutte le coppie di nodi, l'algoritmo seleziona un sottoinsieme casuale di nodi (specificato dal parametro `k`) e calcola la centralità solo per questi nodi.
- Questo metodo riduce significativamente il tempo di calcolo, soprattutto per grafi di grandi dimensioni, a scapito di una minore precisione. La qualità dell'approssimazione dipende dal numero di nodi campionati (`k`): un numero maggiore di campioni migliora l'accuratezza dell'approssimazione.

**Implementazione in NetworkX**:
- Quando si utilizza la funzione `nx.betweenness_centrality()` con il parametro `k` specificato (ad esempio, `k=5`), NetworkX esegue un'approssimazione della betweenness centrality utilizzando il campionamento randomizzato. Il parametro `seed` può essere usato per garantire che i risultati siano riproducibili, utilizzando lo stesso set di campioni ogni volta.

In [27]:
approx_results = []
for i, g in enumerate(graphs):
    betweenness_approx, computation_time = calculate_betweenness_centrality_approximated(g, approximation_rate)
    approx_results.append((betweenness_exact, computation_time))
    print(f"Graph {i+1} - Computation time: {computation_time} seconds")

6386
Graph 1 - Computation time: 81.49726724624634 seconds
18448
Graph 2 - Computation time: 1114.1465520858765 seconds


## Results

In [47]:
from tabulate import tabulate

# Function to get top N nodes by betweenness centrality
def get_top_n_betweenness(betweenness_dict, n):
    return sorted(betweenness_dict.items(), key=lambda item: item[1], reverse=True)[:n]

# Configurable parameter for top N nodes
N = 20  # Change this value as needed

# Prepare and print a table for each graph showing the top N nodes
for i, (exact, approx) in enumerate(zip(exact_results, approx_results)):
    graph_name = f"Graph {i+1}"
    top_n_exact = get_top_n_betweenness(exact[0], N)
    top_n_approx = get_top_n_betweenness(approx[0], N)
    
    top_n_data = []
    for rank, ((node_exact, betw_exact), (node_approx, betw_approx)) in enumerate(zip(top_n_exact, top_n_approx), 1):
        top_n_data.append([rank, node_exact, betw_exact, node_approx, betw_approx])
    
    # Define table headers for top N nodes
    top_n_headers = ["Rank", "Exact Node", "Exact Betweenness", "Approx Node", "Approx Betweenness"]
    
    # Print the top N nodes table for the current graph
    print(f"\n{graph_name} - Top {N} Nodes by Betweenness Centrality")
    print(tabulate(top_n_data, headers=top_n_headers, tablefmt="grid"))


Graph 1 - Top 20 Nodes by Betweenness Centrality
+--------+--------------+---------------------+---------------+----------------------+
|   Rank |   Exact Node |   Exact Betweenness |   Approx Node |   Approx Betweenness |
+========+==============+=====================+===============+======================+
|      1 |          224 |          0.0417423  |          5472 |           0.107291   |
+--------+--------------+---------------------+---------------+----------------------+
|      2 |         4046 |          0.0232396  |         10626 |           0.0579346  |
+--------+--------------+---------------------+---------------+----------------------+
|      3 |         4485 |          0.0128879  |         12499 |           0.0390084  |
+--------+--------------+---------------------+---------------+----------------------+
|      4 |         2442 |          0.0109526  |         14497 |           0.0346414  |
+--------+--------------+---------------------+---------------+-----------------

In [44]:
## Results


# Function to calculate mean squared error manually
def mean_squared_error_manual(y_true, y_pred):
    return sum((yt - yp) ** 2 for yt, yp in zip(y_true, y_pred)) / len(y_true)

# Calculate mean squared error for each graph
mse_results = [
    mean_squared_error_manual(exact[0], approx[0])
    for exact, approx in zip(exact_results, approx_results)
]

# Prepare data for the table
data = []

for i, (exact, approx, mse) in enumerate(zip(exact_results, approx_results, mse_results)):
    graph_name = f"Graph {i+1}"
    exact_time = float(exact[1])
    approx_time = float(approx[1])
    time_diff_percentage = f"{(((exact_time - approx_time) / exact_time) * 100):.2f}%"  # Corrected formatting
    mse_value = mse

    data.append([graph_name, exact_time, approx_time, time_diff_percentage, mse_value])

# Define table headers
headers = ["Graph", "Exact Time (s)", "Approx Time (s)", "Speedup(%)", "MSE"]

# Print the table
print(tabulate(data, headers=headers, tablefmt="grid"))

+---------+------------------+-------------------+--------------+-------------+
| Graph   |   Exact Time (s) |   Approx Time (s) | Speedup(%)   |         MSE |
+=========+==================+===================+==============+=============+
| Graph 1 |          271.908 |           81.4973 | 70.03%       | 6.71647e+07 |
+---------+------------------+-------------------+--------------+-------------+
| Graph 2 |         3621.19  |         1114.15   | 69.23%       | 0           |
+---------+------------------+-------------------+--------------+-------------+
